# Desafio Engenheiro de Dados - Semantix
### Canditado: Denivaldo Lopes Cruz - denivaldo.cruz@gmail.com
### Data: 09/07/2019

## HTTP requests to the NASA Kennedy Space Center WWW server
#### Datasets: NASA_access_log_Jul95.gz e NASA_access_log_Aug95.gz
#### Fonte: http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html

In [1]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import to_date
from pyspark.sql.functions import desc
from pyspark.sql.functions import sum
from pyspark.sql.functions import when
from pyspark.sql.functions import col

import re


In [2]:
#if __name__ == "__main__":
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("DesafioEngenheiroDadosDenivaldoCruz").getOrCreate()

### Carregando os datasets

In [3]:
df = spark.read.text(['NASA_access_log_Jul95.gz', 'NASA_access_log_Aug95.gz'])
df.show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                                 |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                      |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085   |
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0               |
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
+-----------------------

In [18]:
df.head()

Row(value='199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245')

In [4]:
print(df.count())

3461613


In [5]:
df.describe().show()

+-------+--------------------+
|summary|               value|
+-------+--------------------+
|  count|             3461613|
|   mean|                null|
| stddev|                null|
|    min|***.novo.dk - - [...|
|    max|zzzzzzzz.mindspri...|
+-------+--------------------+



### Construindo o dataset com os campos host, timestamp, codigo_retorno e total_bytes

In [6]:
## RegEx dos campos
host_regex = r'(^\S+\.[\S+\.]+\S+)\s'
timestamp_regex = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
codigo_retorno_regex = r'\s(\d{3})\s'
total_bytes_regex = r'\s(\d+)$'

## Extracao dos campos
df_log = df.select(regexp_extract('value', host_regex, 1).alias('host'),
                    regexp_extract('value', timestamp_regex, 1).alias('timestamp'),
                    regexp_extract('value', codigo_retorno_regex, 1).cast('integer').alias('codigo_retorno'),
                    regexp_extract('value', total_bytes_regex, 1).cast('integer').alias('total_bytes'))

df_log.cache()

df_log.show(10, truncate=True)

+--------------------+--------------------+--------------+-----------+
|                host|           timestamp|codigo_retorno|total_bytes|
+--------------------+--------------------+--------------+-----------+
|        199.72.81.55|01/Jul/1995:00:00...|           200|       6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00...|           200|       3985|
|      199.120.110.21|01/Jul/1995:00:00...|           200|       4085|
|  burger.letters.com|01/Jul/1995:00:00...|           304|          0|
|      199.120.110.21|01/Jul/1995:00:00...|           200|       4179|
|  burger.letters.com|01/Jul/1995:00:00...|           304|          0|
|  burger.letters.com|01/Jul/1995:00:00...|           200|          0|
|     205.212.115.106|01/Jul/1995:00:00...|           200|       3985|
|         d104.aa.net|01/Jul/1995:00:00...|           200|       3985|
|      129.94.144.152|01/Jul/1995:00:00...|           200|       7074|
+--------------------+--------------------+--------------+-----------+
only s

### Substituir hosts vazios por 'sem host'

In [7]:
#df_log = df_log.filter("host != ''")

In [8]:
df_log = df_log.withColumn('host', when(col('host') == '', "sem host").otherwise(col('host')))

In [9]:
df_log.filter("host == 'sem host'").show()

+--------+--------------------+--------------+-----------+
|    host|           timestamp|codigo_retorno|total_bytes|
+--------+--------------------+--------------+-----------+
|sem host|01/Jul/1995:04:11...|           200|          0|
|sem host|01/Jul/1995:13:09...|           200|       7074|
|sem host|01/Jul/1995:13:09...|           200|      12169|
|sem host|01/Jul/1995:13:10...|           200|       3666|
|sem host|02/Jul/1995:04:14...|           200|       7074|
|sem host|02/Jul/1995:04:14...|           200|       5866|
|sem host|02/Jul/1995:04:14...|           200|        786|
|sem host|02/Jul/1995:04:14...|           200|        363|
|sem host|02/Jul/1995:04:14...|           200|        234|
|sem host|02/Jul/1995:04:14...|           200|        669|
|sem host|02/Jul/1995:04:16...|           200|       3977|
|sem host|02/Jul/1995:04:16...|           200|      11473|
|sem host|02/Jul/1995:04:16...|           200|       1204|
|sem host|02/Jul/1995:04:19...|           200|       248

In [10]:
## Total de registros
print((df_log.count(), len(df_log.columns)))

(3461613, 4)


## Questões

### 1. Número de hosts únicos: 137933

In [11]:
df_log.agg(countDistinct("host")).show()

+--------------------+
|count(DISTINCT host)|
+--------------------+
|              137933|
+--------------------+



### 2. O total de erros 404: 20899

In [12]:
df_404 = df_log.filter(df_log.codigo_retorno == "404")
df_404.count()

20899

### 3. Os 5 URLs que mais causaram erro 404

#### hoohoo.ncsa.uiuc.edu|  251|
#### piweba3y.prodigy.com|  157|
#### jbiagioni.npt.nuw...|  132|
#### piweba1y.prodigy.com|  114|
#### www-d4.proxy.aol.com|   91|

In [13]:
from pyspark.sql.functions import desc
df_404.groupBy("host").count().sort(desc("count")).show()

+--------------------+-----+
|                host|count|
+--------------------+-----+
|hoohoo.ncsa.uiuc.edu|  251|
|piweba3y.prodigy.com|  157|
|jbiagioni.npt.nuw...|  132|
|piweba1y.prodigy.com|  114|
|            sem host|  112|
|www-d4.proxy.aol.com|   91|
|piweba4y.prodigy.com|   86|
|scooter.pa-x.dec.com|   69|
|www-d1.proxy.aol.com|   64|
|phaelon.ksc.nasa.gov|   64|
|dialip-217.den.mm...|   62|
|www-b4.proxy.aol.com|   62|
|www-b3.proxy.aol.com|   61|
|www-a2.proxy.aol.com|   60|
|piweba2y.prodigy.com|   59|
|www-d2.proxy.aol.com|   59|
|  alyssa.prodigy.com|   56|
|monarch.eng.buffa...|   56|
|www-b2.proxy.aol.com|   53|
|www-c4.proxy.aol.com|   53|
+--------------------+-----+
only showing top 20 rows



### 4. Quantidade de erros 404 por dia

In [14]:
df_404 = df_404.withColumn("dia",to_date(unix_timestamp(df_log["timestamp"], "dd/MMM/yyyy").cast("timestamp")))
df_404.show()

+--------------------+--------------------+--------------+-----------+----------+
|                host|           timestamp|codigo_retorno|total_bytes|       dia|
+--------------------+--------------------+--------------+-----------+----------+
|dd15-062.compuser...|01/Jul/1995:00:01...|           404|       null|1995-07-01|
|   netport-27.iu.net|01/Jul/1995:00:10...|           404|       null|1995-07-01|
|   netport-27.iu.net|01/Jul/1995:00:10...|           404|       null|1995-07-01|
|blv-pm0-ip28.halc...|01/Jul/1995:00:14...|           404|       null|1995-07-01|
|blv-pm0-ip28.halc...|01/Jul/1995:00:14...|           404|       null|1995-07-01|
|cu-dialup-1005.ci...|01/Jul/1995:00:18...|           404|       null|1995-07-01|
|cu-dialup-1005.ci...|01/Jul/1995:00:18...|           404|       null|1995-07-01|
|cu-dialup-1005.ci...|01/Jul/1995:00:18...|           404|       null|1995-07-01|
|    mimas.execpc.com|01/Jul/1995:00:18...|           404|       null|1995-07-01|
|zoom112.telepat

In [15]:
df_404.groupBy('dia').count().orderBy('dia').show(60)

+----------+-----+
|       dia|count|
+----------+-----+
|1995-07-01|  316|
|1995-07-02|  291|
|1995-07-03|  474|
|1995-07-04|  359|
|1995-07-05|  497|
|1995-07-06|  640|
|1995-07-07|  570|
|1995-07-08|  300|
|1995-07-09|  348|
|1995-07-10|  398|
|1995-07-11|  471|
|1995-07-12|  471|
|1995-07-13|  532|
|1995-07-14|  413|
|1995-07-15|  254|
|1995-07-16|  257|
|1995-07-17|  406|
|1995-07-18|  465|
|1995-07-19|  639|
|1995-07-20|  428|
|1995-07-21|  334|
|1995-07-22|  192|
|1995-07-23|  233|
|1995-07-24|  328|
|1995-07-25|  461|
|1995-07-26|  336|
|1995-07-27|  336|
|1995-07-28|   94|
|1995-08-01|  243|
|1995-08-03|  304|
|1995-08-04|  346|
|1995-08-05|  236|
|1995-08-06|  373|
|1995-08-07|  537|
|1995-08-08|  391|
|1995-08-09|  279|
|1995-08-10|  315|
|1995-08-11|  263|
|1995-08-12|  196|
|1995-08-13|  216|
|1995-08-14|  287|
|1995-08-15|  327|
|1995-08-16|  259|
|1995-08-17|  271|
|1995-08-18|  256|
|1995-08-19|  209|
|1995-08-20|  312|
|1995-08-21|  305|
|1995-08-22|  288|
|1995-08-23|

### 5. O total de bytes retornados

In [16]:
df_log.select("total_bytes").groupBy().sum().show()

+----------------+
|sum(total_bytes)|
+----------------+
|     65524314915|
+----------------+



In [17]:
df_log.agg(sum("total_bytes")).show()

+----------------+
|sum(total_bytes)|
+----------------+
|     65524314915|
+----------------+

